# Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import darts
from matplotlib import pyplot as plt

# Data

In [41]:
# Set url:
data_url = (
    "https://github.com/MarcoLandtHayen/climate_index_collection/"
    "releases/download/v2023.03.29.1/climate_indices.csv"
)

# Load data:
climind = pd.read_csv(data_url)

# Format data:
climind = climind.set_index(["model", "year", "month", "index"]).unstack(level=-1)["value"]

# Separate data by model, reset index and drop columns year and month:
climind_FOCI = climind.loc[('FOCI')].reset_index().drop(columns=['year','month'])
climind_CESM = climind.loc[('CESM')].reset_index().drop(columns=['year','month'])

# Extract target series (PREC_SAHEL) and keep ALL indices (including PREC_SAHEL) as inputs:
target_FOCI = climind_FOCI.loc[:, climind_FOCI.columns == 'PREC_SAHEL']
target_CESM = climind_CESM.loc[:, climind_CESM.columns == 'PREC_SAHEL']
input_FOCI = climind_FOCI.loc[:]
input_CESM = climind_CESM.loc[:]

# Normalize target series to have zero mean and unit variance:
target_FOCI = (target_FOCI - np.mean(target_FOCI)) / np.std(target_FOCI)
target_CESM = (target_CESM - np.mean(target_CESM)) / np.std(target_CESM)

# Create targets with lead time 1, 3, 6 months:
target_FOCI_lead1 = target_FOCI[1:-5].values
target_FOCI_lead3 = target_FOCI[3:-3].values
target_FOCI_lead6 = target_FOCI[6:].values
target_CESM_lead1 = target_CESM[1:-5].values
target_CESM_lead3 = target_CESM[3:-3].values
target_CESM_lead6 = target_CESM[6:].values

# Erase last 6 rows from inputs and target, to keep dimensions right:
input_FOCI = input_FOCI[:-6]
input_CESM = input_CESM[:-6]
target_FOCI = target_FOCI[:-6]
target_CESM = target_CESM[:-6]

# Add targets with individual lead times as new columns:
target_FOCI['PREC_SAHEL_lead1'] = target_FOCI_lead1
target_FOCI['PREC_SAHEL_lead3'] = target_FOCI_lead3
target_FOCI['PREC_SAHEL_lead6'] = target_FOCI_lead6
target_CESM['PREC_SAHEL_lead1'] = target_CESM_lead1
target_CESM['PREC_SAHEL_lead3'] = target_CESM_lead3
target_CESM['PREC_SAHEL_lead6'] = target_CESM_lead6

## Set parameters for inputs and target:

# Select input features:
input_features = [
    'AMO', 'ENSO_12', 'ENSO_3', 'ENSO_34', 'ENSO_4', 'NAO_PC', 'NAO_ST', 
    'NP', 'PDO_PC', 'PREC_SAHEL', 'SAM_PC', 'SAM_ZM', 'SAT_N_ALL', 'SAT_N_LAND',
    'SAT_N_OCEAN', 'SAT_S_ALL', 'SAT_S_LAND', 'SAT_S_OCEAN', 'SOI',
    'SSS_ENA', 'SSS_NA', 'SSS_SA', 'SSS_WNA', 'SST_ESIO', 'SST_HMDR',
    'SST_MED', 'SST_TNA', 'SST_TSA', 'SST_WSIO'
]

# Specify relative amount of train data:
train_val_split = 0.9
window_size = 12

## Optionally choose to scale or normalize input features:
# 'no': Keep raw input features.
# 'scale_01': Scale input features with min/max scaling to [0,1].
# 'scale_11': Scale input features with min/max scaling to [-1,1].
# 'norm': Normalize input features, hence subtract mean and divide by std dev.
scale_norm = 'norm'

# Select specified input features:
input_selected_FOCI = input_FOCI[input_features]
input_selected_CESM = input_CESM[input_features]

## Split inputs and targets into train and test sets:

# Get number of train samples:
n_train = int(train_val_split * len(input_selected_FOCI))

# Split inputs and targets:
train_input_FOCI = input_selected_FOCI[:n_train]
train_input_CESM = input_selected_CESM[:n_train]
test_input_FOCI = input_selected_FOCI[n_train - window_size:]
test_input_CESM = input_selected_CESM[n_train - window_size:]

train_target_FOCI = target_FOCI[:n_train]
train_target_CESM = target_CESM[:n_train]
test_target_FOCI = target_FOCI[n_train:]
test_target_CESM = target_CESM[n_train:]

# normalize the data
scaler_FOCI = StandardScaler()
scaler_CESM = StandardScaler()

train_input_scaled_FOCI = scaler_FOCI.fit_transform(train_input_FOCI)
test_input_scaled_FOCI = scaler_FOCI.transform(test_input_FOCI)
train_input_scaled_CESM = scaler_CESM.fit_transform(train_input_CESM)
test_input_scaled_CESM = scaler_CESM.transform(test_input_CESM)

/home/jake/desktop/git/Sahel_Rainfall/.ve_Sahel_Rainfall/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/jake/desktop/git/Sahel_Rainfall/.ve_Sahel_Rainfall/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


# DARTS models
## Linear Regression lead time 1:

In [4]:
# Tryout models
from darts.models import LinearRegressionModel
from darts import TimeSeries
from darts.metrics.metrics import mse

train_input_CESM_ts = TimeSeries.from_values(train_input_scaled_CESM)
train_target_CESM_ts = TimeSeries.from_values(train_target_CESM['PREC_SAHEL_lead1'].values)
test_input_CESM_ts = TimeSeries.from_times_and_values(values=test_input_scaled_CESM, times=pd.RangeIndex(10794-window_size, 11982))
test_target_CESM_ts = TimeSeries.from_times_and_values(values=test_target_CESM['PREC_SAHEL_lead1'].values, times=pd.RangeIndex(10794, 11982))

naive_model = LinearRegressionModel(lags_past_covariates=window_size)
naive_model.fit(series=train_target_CESM_ts, past_covariates=train_input_CESM_ts)

predictions = naive_model.predict(1188, past_covariates=test_input_CESM_ts)

mse_lead1 = mse(actual_series=test_target_CESM_ts, pred_series=predictions)
print(f"The MSE for lead time 1 is: {mse_lead1}")


The MSE for lead time 1 is: 0.9991140622468313


## Linear Regression lead time 3:

In [5]:
train_input_CESM_ts = TimeSeries.from_values(train_input_scaled_CESM)
train_target_CESM_ts = TimeSeries.from_values(train_target_CESM['PREC_SAHEL_lead3'].values)
test_input_CESM_ts = TimeSeries.from_times_and_values(values=test_input_scaled_CESM, times=pd.RangeIndex(10794-window_size, 11982))
test_target_CESM_ts = TimeSeries.from_times_and_values(values=test_target_CESM['PREC_SAHEL_lead3'].values, times=pd.RangeIndex(10794, 11982))

naive_model = LinearRegressionModel(lags_past_covariates=window_size)
naive_model.fit(series=train_target_CESM_ts, past_covariates=train_input_CESM_ts)

predictions = naive_model.predict(1188, past_covariates=test_input_CESM_ts)

mse_lead3 = mse(actual_series=test_target_CESM_ts, pred_series=predictions)
print(f"The MSE for lead time 3 is: {mse_lead3}")

The MSE for lead time 3 is: 1.0328763529151652


## Linear Regression lead time 6:

In [6]:
train_input_CESM_ts = TimeSeries.from_values(train_input_scaled_CESM)
train_target_CESM_ts = TimeSeries.from_values(train_target_CESM['PREC_SAHEL_lead6'].values)
test_input_CESM_ts = TimeSeries.from_times_and_values(values=test_input_scaled_CESM, times=pd.RangeIndex(10794-window_size, 11982))
test_target_CESM_ts = TimeSeries.from_times_and_values(values=test_target_CESM['PREC_SAHEL_lead6'].values, times=pd.RangeIndex(10794, 11982))

naive_model = LinearRegressionModel(lags_past_covariates=window_size)
naive_model.fit(series=train_target_CESM_ts, past_covariates=train_input_CESM_ts)

predictions = naive_model.predict(1188, past_covariates=test_input_CESM_ts)

mse_lead6 = mse(actual_series=test_target_CESM_ts, pred_series=predictions)
print(f"The MSE for lead time 6 is: {mse_lead6}")

The MSE for lead time 6 is: 1.0535867615718735


## XG Boost model

In [11]:
from darts.models.forecasting.xgboost import XGBModel

train_input_CESM_ts = TimeSeries.from_values(train_input_scaled_CESM)
train_target_CESM_ts = TimeSeries.from_values(train_target_CESM['PREC_SAHEL_lead1'].values)
test_input_CESM_ts = TimeSeries.from_times_and_values(values=test_input_scaled_CESM, times=pd.RangeIndex(10794-window_size, 11982))
test_target_CESM_ts = TimeSeries.from_times_and_values(values=test_target_CESM['PREC_SAHEL_lead1'].values, times=pd.RangeIndex(10794, 11982))


model_xgb = XGBModel(lags_past_covariates=window_size, output_chunk_length=1)
model_xgb.fit(series=train_target_CESM_ts, past_covariates=train_input_CESM_ts)

predictions = model_xgb.predict(1188, past_covariates=test_input_CESM_ts)

mse_lead1 = mse(actual_series=test_target_CESM_ts, pred_series=predictions)
print(f"The MSE for lead time 1 is: {mse_lead1}")

The MSE for lead time 1 is: 1.0972040109315113


In [26]:
from darts.models.forecasting.block_rnn_model import BlockRNNModel

train_input_CESM_ts = TimeSeries.from_values(train_input_scaled_CESM)
train_target_CESM_ts = TimeSeries.from_values(
    train_target_CESM["PREC_SAHEL_lead1"].values
)
test_input_CESM_ts = TimeSeries.from_times_and_values(
    values=test_input_scaled_CESM, times=pd.RangeIndex(10794 - window_size, 11982)
)
test_target_CESM_ts = TimeSeries.from_times_and_values(
    values=test_target_CESM["PREC_SAHEL_lead1"].values,
    times=pd.RangeIndex(10794, 11982),
)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# stop training when validation loss does not decrease more than 0.05 (`min_delta`) over
# a period of 5 epochs (`patience`)
my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=5,
    min_delta=0.05,
    mode='min',
)

pl_trainer_kwargs={"callbacks": [my_stopper]}

model_RNN = BlockRNNModel(
    input_chunk_length=window_size,
    output_chunk_length=1,
    model="LSTM",
    hidden_dim=32,
    n_rnn_layers=3,
    dropout=0.3,
    pl_trainer_kwargs=pl_trainer_kwargs,
    log_tensorboard=True,

)

model_RNN.fit(series=train_target_CESM_ts, past_covariates=train_input_CESM_ts, val_series=test_target_CESM_ts, val_past_covariates=test_input_CESM_ts)

predictions = model_RNN.predict(1188, past_covariates=test_input_CESM_ts)

mse_lead1 = mse(actual_series=test_target_CESM_ts, pred_series=predictions)
print(f"The MSE for lead time 1 is: {mse_lead1}")



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 25.1 K
4 | fc            | Sequential       | 33    
---------------------------------------------------
25.1 K    Trainable params
0         Non-trainable params


Epoch 7: 100%|██████████| 337/337 [00:02<00:00, 133.74it/s, v_num=logs, train_loss=0.520, val_loss=0.946]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.88it/s]
The MSE for lead time 1 is: 0.9477404867715031


In [30]:
from darts.models.forecasting.block_rnn_model import BlockRNNModel

train_input_CESM_ts = TimeSeries.from_values(train_input_scaled_CESM)
train_target_CESM_ts = TimeSeries.from_values(
    train_target_CESM["PREC_SAHEL_lead1"].values
)
test_input_CESM_ts = TimeSeries.from_times_and_values(
    values=test_input_scaled_CESM, times=pd.RangeIndex(10794 - window_size, 11982)
)
test_target_CESM_ts = TimeSeries.from_times_and_values(
    values=test_target_CESM["PREC_SAHEL_lead1"].values,
    times=pd.RangeIndex(10794, 11982),
)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# stop training when validation loss does not decrease more than 0.05 (`min_delta`) over
# a period of 5 epochs (`patience`)
my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=5,
    min_delta=0.01,
    mode='min',
)

pl_trainer_kwargs={"callbacks": [my_stopper]}

model_RNN = BlockRNNModel(
    input_chunk_length=window_size,
    output_chunk_length=1,
    model="LSTM",
    hidden_dim=64,
    n_rnn_layers=3,
    dropout=0.3,
    pl_trainer_kwargs=pl_trainer_kwargs,
    log_tensorboard=True,

)

model_RNN.fit(series=train_target_CESM_ts, past_covariates=train_input_CESM_ts, val_series=test_target_CESM_ts, val_past_covariates=test_input_CESM_ts)

predictions = model_RNN.predict(1188, past_covariates=test_input_CESM_ts)

mse_lead1 = mse(actual_series=test_target_CESM_ts, pred_series=predictions)
print(f"The MSE for lead time 1 is: {mse_lead1}")



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 91.1 K
4 | fc            | Sequential       | 65    
---------------------------------------------------
91.2 K    Trainable params
0         Non-trainable params


Epoch 7: 100%|██████████| 337/337 [00:03<00:00, 106.44it/s, v_num=logs, train_loss=0.991, val_loss=0.967]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.61it/s]
The MSE for lead time 1 is: 0.9629192905164679


In [35]:
from darts.models.forecasting.transformer_model import TransformerModel

train_input_CESM_ts = TimeSeries.from_values(train_input_scaled_CESM)
train_target_CESM_ts = TimeSeries.from_values(
    train_target_CESM["PREC_SAHEL_lead1"].values
)
test_input_CESM_ts = TimeSeries.from_times_and_values(
    values=test_input_scaled_CESM, times=pd.RangeIndex(10794 - window_size, 11982)
)
test_target_CESM_ts = TimeSeries.from_times_and_values(
    values=test_target_CESM["PREC_SAHEL_lead1"].values,
    times=pd.RangeIndex(10794, 11982),
)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# stop training when validation loss does not decrease more than 0.05 (`min_delta`) over
# a period of 5 epochs (`patience`)
my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=7,
    min_delta=0.01,
    mode='min',
)

pl_trainer_kwargs={"callbacks": [my_stopper]}

model_trans = TransformerModel(
    input_chunk_length=window_size,
    output_chunk_length=1,
    dropout=0.3,
    pl_trainer_kwargs=pl_trainer_kwargs,
    log_tensorboard=True,

)

model_trans.fit(series=train_target_CESM_ts, past_covariates=train_input_CESM_ts, val_series=test_target_CESM_ts, val_past_covariates=test_input_CESM_ts)

predictions = model_trans.predict(1188, past_covariates=test_input_CESM_ts)

mse_lead1 = mse(actual_series=test_target_CESM_ts, pred_series=predictions)
print(f"The MSE for lead time 1 is: {mse_lead1}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type                | Params
------------------------------------------------------------
0 | criterion           | MSELoss             | 0     
1 | train_metrics       | MetricCollection    | 0     
2 | val_metrics         | MetricCollection    | 0     
3 | encoder             | Linear              | 2.0 K 
4 | positional_encoding | _PositionalEncoding | 0     
5 | transformer         | Transformer         |

Epoch 10:  74%|███████▎  | 246/334 [00:13<00:04, 18.79it/s, v_num=logs, train_loss=0.378, val_loss=1.060]

/home/jake/desktop/git/Sahel_Rainfall/.ve_Sahel_Rainfall/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:01<00:00,  1.92s/it]
The MSE for lead time 1 is: 1.0407771839207207


In [39]:
from darts.models.forecasting.nbeats import NBEATSModel

train_input_CESM_ts = TimeSeries.from_values(train_input_scaled_CESM)
train_target_CESM_ts = TimeSeries.from_values(
    train_target_CESM["PREC_SAHEL_lead1"].values
)
test_input_CESM_ts = TimeSeries.from_times_and_values(
    values=test_input_scaled_CESM, times=pd.RangeIndex(10794 - window_size, 11982)
)
test_target_CESM_ts = TimeSeries.from_times_and_values(
    values=test_target_CESM["PREC_SAHEL_lead1"].values,
    times=pd.RangeIndex(10794, 11982),
)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# stop training when validation loss does not decrease more than 0.05 (`min_delta`) over
# a period of 5 epochs (`patience`)
my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=7,
    min_delta=0.01,
    mode='min',
)

pl_trainer_kwargs={"callbacks": [my_stopper]}

model_nbeats = NBEATSModel(
    input_chunk_length=window_size,
    output_chunk_length=1,
    dropout=0.3,
    pl_trainer_kwargs=pl_trainer_kwargs,
    log_tensorboard=True,

)

model_nbeats.fit(series=train_target_CESM_ts, past_covariates=train_input_CESM_ts, val_series=test_target_CESM_ts, val_past_covariates=test_input_CESM_ts)

predictions = model_nbeats.predict(1188, past_covariates=test_input_CESM_ts)

mse_lead1 = mse(actual_series=test_target_CESM_ts, pred_series=predictions)
print(f"The MSE for lead time 1 is: {mse_lead1}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 34.3 M
---------------------------------------------------
34.3 M    Trainable params
22.9 K    Non-trainable params
34.3 M    Total params
137.230   Total estimat

Epoch 7: 100%|██████████| 334/334 [00:28<00:00, 11.60it/s, v_num=logs, train_loss=571.0, val_loss=1.09e+5]    


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:05<00:00,  5.39s/it]
The MSE for lead time 1 is: 22.32790993366191


In [42]:
from darts.models.forecasting.tft_model import TFTModel

train_input_CESM_ts = TimeSeries.from_values(train_input_scaled_CESM)
train_target_CESM_ts = TimeSeries.from_values(
    train_target_CESM["PREC_SAHEL_lead1"].values
)
test_input_CESM_ts = TimeSeries.from_times_and_values(
    values=test_input_scaled_CESM, times=pd.RangeIndex(10794 - window_size, 11982)
)
test_target_CESM_ts = TimeSeries.from_times_and_values(
    values=test_target_CESM["PREC_SAHEL_lead1"].values,
    times=pd.RangeIndex(10794, 11982),
)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# stop training when validation loss does not decrease more than 0.05 (`min_delta`) over
# a period of 5 epochs (`patience`)
my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=5,
    min_delta=0.01,
    mode='min',
)

pl_trainer_kwargs={"callbacks": [my_stopper]}

model_tft = TFTModel(
    input_chunk_length=window_size,
    output_chunk_length=1,
    dropout=0.3,
    pl_trainer_kwargs=pl_trainer_kwargs,
    log_tensorboard=True,
    add_relative_index=True

)

model_tft.fit(series=train_target_CESM_ts, past_covariates=train_input_CESM_ts, val_series=test_target_CESM_ts, val_past_covariates=test_input_CESM_ts)

predictions = model_tft.predict(1188, past_covariates=test_input_CESM_ts)

mse_lead1 = mse(actual_series=test_target_CESM_ts, pred_series=predictions)
print(f"The MSE for lead time 1 is: {mse_lead1}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covar

Epoch 21: 100%|██████████| 337/337 [00:12<00:00, 26.38it/s, v_num=logs, train_loss=2.610, val_loss=3.190]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:07<00:00,  7.96s/it]
The MSE for lead time 1 is: 1.7924541733682364
